In [1]:
import sys
import random
import string
import os
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QPushButton, QComboBox, QSlider, QMessageBox, QInputDialog, QDialog, QTextEdit, QLineEdit, QFileDialog

from PyQt5.QtCore import Qt
import nacl.secret
import nacl.utils

from password_strength import PasswordStats

class PasswordManager(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        
    def initUI(self):
        self.setWindowTitle('Password Manager')
        layout = QVBoxLayout()

        # Create widgets
        self.total_characters_label = QLabel('Total Characters:')
        self.total_characters_slider = QSlider(Qt.Horizontal)
        self.total_characters_slider.setMinimum(1)
        self.total_characters_slider.setMaximum(50)  # Maximum number of characters changed to 50
        self.total_characters_slider.setValue(1)  # Initial value of the slider
        self.total_characters_display = QLabel('1')  # Initial value of the total characters
        self.special_characters_label = QLabel('Number of Special Characters:')
        self.special_characters_input = QComboBox()  # Dropdown for selecting number of special characters
        self.special_characters_input.addItems([str(i) for i in range(0, 11)])  # Options 0-10
        self.number_label = QLabel('Number of Numbers:')
        self.number_input = QComboBox()  # Dropdown for selecting number of numbers
        self.number_input.addItems([str(i) for i in range(0, 11)])  # Options 0-10
        self.capital_letters_label = QLabel('Include Capital Letters?')
        self.capital_letters_input = QComboBox()
        self.capital_letters_input.addItems(['Yes', 'No'])
        self.generate_button = QPushButton('Generate Password')
        self.strength_label = QLabel('Password Strength:')
        self.password_label = QLabel('Generated Password: ')
        self.encrypted_label = QLabel('Encrypted Password: ')
        self.secret_key_label = QLabel('Secret Key: ')
        self.decrypt_button = QPushButton('I want to decrypt')

        # Add widgets to layout
        layout.addWidget(self.total_characters_label)
        layout.addWidget(self.total_characters_slider)
        layout.addWidget(self.total_characters_display)
        layout.addWidget(self.special_characters_label)
        layout.addWidget(self.special_characters_input)
        layout.addWidget(self.number_label)
        layout.addWidget(self.number_input)
        layout.addWidget(self.capital_letters_label)
        layout.addWidget(self.capital_letters_input)
        layout.addWidget(self.generate_button)
        layout.addWidget(self.strength_label)
        layout.addWidget(self.password_label)
        layout.addWidget(self.encrypted_label)
        layout.addWidget(self.secret_key_label)
        layout.addWidget(self.decrypt_button)

        # Connect slider value changed to update_display function
        self.total_characters_slider.valueChanged.connect(self.update_total_characters_display)

        # Connect button click to generate_password function
        self.generate_button.clicked.connect(self.generate_password)

        # Connect button click to decrypt_password function
        self.decrypt_button.clicked.connect(self.decrypt_password)

        # Set layout
        self.setLayout(layout)

    def update_total_characters_display(self, value):
        # Update the label to display the current value of the total characters slider
        self.total_characters_display.setText(str(value))

    def generate_password(self):
        total_characters = int(self.total_characters_display.text())  # Convert the displayed value to an integer
        special_characters_count = int(self.special_characters_input.currentText())
        number_count = int(self.number_input.currentText())
        allow_special_chars = special_characters_count > 0
        allow_numbers = number_count > 0
        include_capital_letters = self.capital_letters_input.currentText() == 'Yes'

        if special_characters_count + number_count > total_characters:
            QMessageBox.warning(self, 'Error', 'Number of special characters and numbers cannot exceed total characters!')
            return

        # Generate password
        password, secret_key = self.generate_password_helper(total_characters, special_characters_count, number_count, allow_special_chars, allow_numbers, include_capital_letters)

        # Encrypt the password
        encrypted_password = self.encrypt_password(password, secret_key)

        # Display the generated password
        self.password_label.setText(f'Generated Password: {password}')
        
        # Display the encrypted password in chunks of 50 characters per line
        encrypted_password_chunks = [encrypted_password[i:i+50] for i in range(0, len(encrypted_password), 50)]
        encrypted_password_display = '\n'.join(encrypted_password_chunks)
        self.encrypted_label.setText(f'Encrypted Password:\n{encrypted_password_display}')

        # Display the secret key
        self.secret_key_label.setText(f'Secret Key: {secret_key.hex()}')

        # Evaluate password strength
        strength_score = PasswordStats(password).strength() * 100
        self.strength_label.setText(f'Password Strength: {strength_score:.2f} / 100')

        # Ask user for permission to save the password
        confirmation = QMessageBox.question(self, 'Save Password', 'Do you want to save the encrypted password?', QMessageBox.Yes | QMessageBox.No)
        if confirmation == QMessageBox.Yes:
            self.save_password(encrypted_password, secret_key)

    def generate_password_helper(self, length, special_characters_count, number_count, allow_special_chars, allow_numbers, include_capital_letters):
        file_path = os.path.join(os.path.expanduser('~'), 'Documents', 'Password', 'password.lst')
        with open(file_path, 'r') as file:
            words = [line.strip() for line in file]

        length -= special_characters_count + number_count
        
        password = ''.join(random.choices(words, k=length))
        
        # Ensure that the password length matches the desired length
        password = password[:length]

        # Maintain a set of used indexes for special characters
        used_indexes = set()

        # Generate the special characters and insert them at random positions in the password
        if allow_special_chars:
            for _ in range(special_characters_count):
                special_char = random.choice(string.punctuation)
                insert_index = random.randint(0, length - 1)  # Generate a random index within the valid range
                while insert_index in used_indexes:
                    insert_index = random.randint(0, length - 1)  # Generate a new random index if already used
                password = password[:insert_index] + special_char + password[insert_index:]
                used_indexes.add(insert_index)

        # Generate the numbers and insert them at random positions in the password
        if allow_numbers:
            for _ in range(number_count):
                number = random.choice(string.digits)
                insert_index = random.randint(0, length - 1)  # Generate a random index within the valid range
                while insert_index in used_indexes:
                    insert_index = random.randint(0, length - 1)  # Generate a new random index if already used
                password = password[:insert_index] + number + password[insert_index:]
                used_indexes.add(insert_index)

        # Convert password to lowercase if capital letters are not allowed
        if not include_capital_letters:
            password = password.lower()

        # If capital letters allowed, ensure at least 1 letter is capitalized
        if include_capital_letters:
            capital_index = random.randint(0, length - 1)  # Generate a random index within the password length
            password = password[:capital_index] + password[capital_index].upper() + password[capital_index + 1:]
        
        # Generate a secret key for encryption
        secret_key = nacl.utils.random(nacl.secret.SecretBox.KEY_SIZE)

        return password, secret_key

    def encrypt_password(self, password, secret_key):
        # Initialize a secret box with the key
        box = nacl.secret.SecretBox(secret_key)

        # Encrypt the password
        encrypted_password = box.encrypt(password.encode('utf-8'))

        return encrypted_password.hex()

    def save_password(self, encrypted_password, secret_key):
        # Create the Password folder if it doesn't exist
        folder_path = os.path.join(os.path.expanduser('~'), 'Documents', 'Password')
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        # Ask user for the filename to save the password
        file_path, _ = QFileDialog.getSaveFileName(self, 'Save Encrypted Password', folder_path, 'Text Files (*.txt)')
        if file_path:
            # Write the encrypted password and secret key to the file
            with open(file_path, 'w') as file:
                file.write('Encrypted Password: ')
                file.write(encrypted_password)
                file.write('\n\n')  # Empty line
                file.write('Secret Key: ')
                file.write(secret_key.hex())

    def decrypt_password(self):
        # Open a new window to input the encrypted password and secret key
        dialog = DecryptDialog()
        dialog.exec_()

class DecryptDialog(QDialog):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Decrypt Password')
        self.layout = QVBoxLayout()

        self.encrypted_label = QLabel('Enter Encrypted Password:')
        self.encrypted_input = QTextEdit()
        self.secret_key_label = QLabel('Enter Secret Key:')
        self.secret_key_input = QLineEdit()
        self.decrypt_button = QPushButton('Decrypt')
        self.decrypt_output = QLineEdit()

        self.layout.addWidget(self.encrypted_label)
        self.layout.addWidget(self.encrypted_input)
        self.layout.addWidget(self.secret_key_label)
        self.layout.addWidget(self.secret_key_input)
        self.layout.addWidget(self.decrypt_button)
        self.layout.addWidget(self.decrypt_output)

        self.decrypt_button.clicked.connect(self.decrypt)

        self.setLayout(self.layout)

    def decrypt(self):
        # Get the encrypted password and secret key from the input fields
        encrypted_password = self.encrypted_input.toPlainText().strip()
        secret_key_hex = self.secret_key_input.text().strip()

        # Convert secret key from hex string to bytes
        try:
            secret_key = bytes.fromhex(secret_key_hex)
        except ValueError:
            self.decrypt_output.setText('Invalid secret key format!')
            return

        # Initialize a secret box with the key
        box = nacl.secret.SecretBox(secret_key)

        # Decrypt the password
        try:
            decrypted_password = box.decrypt(bytes.fromhex(encrypted_password))
            self.decrypt_output.setText(f'Decrypted Password: {decrypted_password.decode("utf-8")}')
        except:
            self.decrypt_output.setText('Decryption failed! Invalid secret key or encrypted password.')

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = PasswordManager()
    window.show()
    try:
        sys.exit(app.exec_())
    except SystemExit:
        pass